In [1]:
from bs4 import BeautifulSoup
from requests import get

In [2]:
#Let website think that we are actual user
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [76]:
#Sample rentals.ca website
rentals = "https://rentals.ca/toronto?p=3"

In [77]:
response = get(rentals, headers=headers)
#200 is a good response
print(response)

<Response [200]>


In [78]:
print(response.text[:1000])

<!doctype html>





<html lang="en">

<head>
  <meta charset="utf-8">

  

<link rel="preconnect" href="https://use.typekit.net">
<link rel="preconnect" href="https://images.rentals.ca">
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://maps.googleapis.com">
<link rel="preconnect" href="https://maps.gstatic.com">
<link rel="preconnect" href="https://www.google-analytics.com">
<link rel="preconnect" href="https://s7.addthis.com">
<link rel="preconnect" href="https://cdnjs.cloudflare.com">
<link rel="preconnect" href="https://static.hotjar.com">
<link rel="preconnect" href="https://script.hotjar.com">
<link rel="preconnect" href="https://vars.hotjar.com">
<link rel="preconnect" href="https://cdn.sift.com">
<link rel="preconnect" href="https://hexagon-analytics.com">
<link rel="preconnect" href="https://analytics.trovit.com">

<link rel="preload" as="script" href="/static/rentalsca-phoenix/js/app.a5cc35a02ae9.js">


  <meta http-equiv="x-ua-

In [79]:
html_soup = BeautifulSoup(response.text, 'html.parser')

all_script = html_soup.find_all('script')

json_list = []
for item in all_script:
    if str(item).startswith('''<script type="applic'''):
        json_list.append(str(item))

In [80]:
dic = {'name':[], 'price': [], 'latitude': [], 'longitude': [], 'url': [], 'location': [], 'room_type': []}
for item in json_list:
    i = 1
    j = 1
    temp_room_type = []
    temp_price = []
    for words in item.split('\n'):
        if words.strip()[:6] == '''"name"''':
            if i == 1:
                dic['name'].append(words.strip()[7:-1].strip().strip('\"'))
                i += 1
            elif i == 2:
                dic['location'].append(''.join(e for e in words.strip()[7:-1] if e.isalnum()))
                i += 1
            else:
                temp_room_type.append(words.strip()[7:-1].strip().strip('\"'))
        if words.strip()[:7] == '''"price"''':
            temp_price.append(float(words.strip()[8:].strip()[:-1]))
        if words.strip()[:10] == '''"latitude"''':
            dic['latitude'].append(float(words.strip()[11:].strip()[:-1]))
        if words.strip()[:11] == '''"longitude"''':
            dic['longitude'].append(float(words.strip()[12:].strip()[:-1]))
        if words.strip()[:5] == '''"url"''' and j == 1:
            dic['url'].append(words.strip()[6:-1].strip().strip('\"'))
            j += 1
    
    #Take out those that do not have room type and price
    if len(temp_room_type) == 0 and len(temp_price) == 0:
        if len(dic['name']) != 0:
            dic['name'].pop()
    else:
        dic['room_type'].append(temp_room_type)
        dic['price'].append(temp_price)

In [93]:
def scrapping_rentals_ca(pagenum = 50):
    #Initialize the final dict for return
    dic = {'name':[], 'price': [], 'latitude': [], 'longitude': [], 'url': [], 'location': [], 'room_type': []}
    #Initialize the link
    initial_link = 'https://rentals.ca/toronto?p='
    #Go to the specificied link page and get the housing info
    for idx in range(pagenum):
        link = initial_link + str(idx+1)
        response = get(link, headers=headers)
        html_soup = BeautifulSoup(response.text, 'html.parser')

        all_script = html_soup.find_all('script')

        json_list = []
        for item in all_script:
            if str(item).startswith('''<script type="applic'''):
                json_list.append(str(item))
        
        #For each webpage, save its info in a temp list and then add to dictionary
        for item in json_list:
            i = 1
            j = 1
            temp_room_type = []
            temp_price = []
            temp_name = []
            for words in item.split('\n'):
                if words.strip()[:6] == '''"name"''':
                    if i == 1:
                        temp_name.append(words.strip()[7:-1].strip().strip('\"'))
                        i += 1
                    elif i == 2:
                        dic['location'].append(''.join(e for e in words.strip()[7:-1] if e.isalnum()))
                        i += 1
                    else:
                        temp_room_type.append(words.strip()[7:-1].strip().strip('\"'))
                if words.strip()[:7] == '''"price"''':
                    temp_price.append(float(words.strip()[8:].strip()[:-1]))
                if words.strip()[:10] == '''"latitude"''':
                    dic['latitude'].append(float(words.strip()[11:].strip()[:-1]))
                if words.strip()[:11] == '''"longitude"''':
                    dic['longitude'].append(float(words.strip()[12:].strip()[:-1]))
                if words.strip()[:5] == '''"url"''' and j == 1:
                    dic['url'].append(words.strip()[6:-1].strip().strip('\"'))
                    j += 1
            if len(temp_room_type) == 0 and len(temp_price) == 0:
                if len(temp_name) != 0:
                    temp_name.pop()
            else:
                dic['name'].append(temp_name)
                dic['room_type'].append(temp_room_type)
                dic['price'].append(temp_price)
                #print("Iteration {}, dictionary length now is {}".format(idx, [len(x) for x in dic.values()]))
    return dic

In [94]:
scrapped = scrapping_rentals_ca(pagenum=150)

In [58]:
csv_columns = list(k for k in dic.keys())

In [59]:
csv_columns

['name', 'price', 'latitude', 'longitude', 'url', 'location', 'room_type']

In [95]:
[len(x) for x in scrapped.values()]

[1500, 1500, 1500, 1500, 1500, 1500, 1500]

In [63]:
import pandas as pd

In [96]:
scrapped_df = pd.DataFrame.from_dict(scrapped)

In [97]:
scrapped_df.head()

,name,price,latitude,longitude,url,location,room_type
0,[120 Raglan Avenue],"[1864.0, 2003.0, 2347.0]",43.686516,-79.421005,https://rentals.ca/york/120-raglan-avenue,York,"[Jr. 1 Bedroom, 1 Bedroom, 2 Bedroom]"
1,[310-312 Lonsdale Road],"[2134.0, 2475.0]",43.689224,-79.411199,https://rentals.ca/toronto/310-312-lonsdale-road,Toronto,"[1 Bedroom, 2 Bedroom]"
2,[35 Walmer Road],"[2183.0, 2353.0, 2888.0]",43.668877,-79.405555,https://rentals.ca/toronto/35-walmer-road,Toronto,"[Jr. 1 Bedroom, 1 Bedroom, 2 Bedroom]"
3,[50 Spadina Road],"[2549.0, 2732.0, 3506.0]",43.669070,-79.404672,https://rentals.ca/toronto/50-spadina-road,Toronto,"[1 Bedroom, Large 1-Bedroom, 2 Bedroom]"
4,[45 Carlton Street],[3700.0],43.661415,-79.380663,https://rentals.ca/toronto/45-carlton-street-2,Toronto,[2.5 Bedroom]


In [98]:
scrapped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
name         1500 non-null object
price        1500 non-null object
latitude     1500 non-null float64
longitude    1500 non-null float64
url          1500 non-null object
location     1500 non-null object
room_type    1500 non-null object
dtypes: float64(2), object(5)
memory usage: 82.1+ KB


In [99]:
scrapped_df.to_csv('rental.csv', index=False)